In [1]:
from sqlalchemy import create_engine, text
import pandas as pd
# Connect to the source SQL Server database
Main_server_name1='DESKTOP-DRB7DGT\danya'
Main_DB1='Bikestores'
Main_DB2='Data'
user=''
password=''

source_engine = create_engine(
                    f"mssql+pyodbc://{user}:{password}@{Main_server_name1}/{Main_DB1}?driver=ODBC+Driver+17+for+SQL+Server",
                    fast_executemany=True)


destination_engine = create_engine(
                    f"mssql+pyodbc://{user}:{password}@{Main_server_name1}/{Main_DB2}?driver=ODBC+Driver+17+for+SQL+Server",
                    fast_executemany=True)

    



def Table_names(text):
    df = pd.read_sql(text, source_engine)
    df["table"] = df["schema_names"] +"."+ df["table_names"]
    df.drop([df.columns[0], df.columns[1]],inplace=True, axis=1)
    return Migration(df)


def Migration(df):
    
    
    total=0
    for table in df["table"]:
        
    
        # Read the data from the source SQL Server table and load it into a Pandas dataframe
        df = pd.read_sql(f"SELECT * FROM {table}", source_engine)
        schema_name=table.split(".")[0]
        table_name=table.split(".")[1]
        # Write the dataframe to the destination SQL Server table
        df.to_sql(table_name, destination_engine, schema=schema_name,if_exists='replace',index=False)
        total=total+ len(df)
    
    print(total ,"rows have been migrated successfully")

    
# those are tables I want to extract from sql servers
   
text='''select s.name as schema_names,t.name as table_names from sys.schemas s join sys.tables t
        on s.schema_id=t.schema_id
        where s.name in('production','sales') and t.name not in('contacts','product_audits','promotions')'''    
Table_names(text)


9077 rows have been migrated successfully
